# Refugees in the Age of Gloabl Warming
### Data Engineering Capstone Project

#### Project Summary
This project focuses on monitoring refugee and population information around the world based on temperature changes over time.

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Imports and installs
import pandas as pd
import os
from datetime import datetime
import psycopg2
import configparser
config = configparser.ConfigParser()
from sqlalchemy.engine import create_engine
# Source: https://github.com/konstantinstadler/country_converter
import country_converter as coco
from pandasql import sqldf
# Source: https://github.com/unitedstates/python-us
import us
# pip3 install -U country_converter 
# pip3 install -U pandasql 
# pip3 install -U us

### Step 1: Scope the Project and Gather Data

#### Scope 
What is your end solution look like? What tools did you use? etc>

The plan is to build a data warehouse for analytical processes, so analysts can design recurring and ad hoc reports over time using SQL. There is a strong emphasis in ensuring the warehouse is easy to interpret, performant, and quality assured.
 
#### Data Sources and Content

There are four source datasets:
 1. City_temperature.csv
     - Summary: average daily temperature for all major cities in the world from 1995 - 2020
     - Source: University of Dayton - separate txt files available for each city [here](https://academic.udayton.edu/kissock/http/Weather/default.htm). The data is available for research and non-commercial purposes only. Refer to [this page](https://academic.udayton.edu/kissock/http/Weather/default.htm) for license.
     - Secondary source: SRK via Kaggle - [link](https://www.kaggle.com/sudalairajkumar/daily-temperature-of-major-cities)
 2. Country_population_total_long.csv
     - Summary: annual population counts by country from 1960 - 2017
     - Source: The World Bank - [link](https://data.worldbank.org/indicator/SP.POP.TOTL)
     - Secondary source: Devakumar kp via Kaggle - [link](https://www.kaggle.com/imdevskp/world-population-19602018?select=population_total_long.csv)
 3. UNdata_City_Population_20210315.csv
     - Summary: annual population counts by city from 1970 - 2020 (contains gaps in 1970's)
     - Source: UN Data - [link](https://data.un.org/Data.aspx?d=POP&f=tableCode%3A240)
 4. UNdata_Refugees_20210217.csv
     - Summary: annual refugee counts from 1975 - 2016 by country of residence and country of origin
     - Source: UN Data - [link](http://data.un.org/Data.aspx?d=UNHCR&f=indID%3aType-Ref)

### Read in Each Dataset

#### Temperature Data

In [2]:
temp_df = pd.read_csv('Data/temperature_data/city_temperature.csv', engine = 'python')
temp_df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


#### Population Counts by Country and Year

In [3]:
country_pop_df = pd.read_csv('Data/country_population_data/country_population_total_long.csv', engine = 'python')
country_pop_df.head()

,Country Name,Year,Count
0,Aruba,1960,54211
1,Afghanistan,1960,8996973
2,Angola,1960,5454933
3,Albania,1960,1608800
4,Andorra,1960,13411


#### Population Counts by City and Year

In [4]:
city_pop_df = pd.read_csv('Data/city_population_data/UNdata_City_Population_20210315.csv', engine = 'python')
city_pop_df.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2019,Total,Both Sexes,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,11711.0,1
1,Åland Islands,2019,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,5606.0,1
2,Åland Islands,2019,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2020,6105.0,1
3,Åland Islands,2018,Total,Both Sexes,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2019,11709.0,1
4,Åland Islands,2018,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2019,5620.5,1


#### Refugee Counts by Year, Country of Residence, and Country of Origin

In [5]:
refugee_df = pd.read_csv('Data/refugee_data/UNdata_Refugees_20210317.csv', engine = 'python')
refugee_df.head()

,Country or territory of asylum or residence,Country or territory of origin,Year,Refugees,Refugees assisted by UNHCR,Total refugees and people in refugee-like situations,Total refugees and people in refugee-like situations assisted by UNHCR
0,Afghanistan,Iraq,2016,1.0,1.0,1.0,1.0
1,Afghanistan,Islamic Rep. of Iran,2016,33.0,33.0,33.0,33.0
2,Afghanistan,Pakistan,2016,59737.0,59737.0,59737.0,59737.0
3,Albania,China,2016,11.0,11.0,11.0,11.0
4,Albania,Dem. Rep. of the Congo,2016,3.0,3.0,3.0,3.0


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### Temperature Data

In [6]:
# Add a date field
temp_df['Date'] = temp_df['Month'].map(str) + '/' + \
                  temp_df['Day'].map(str) + '/' +  \
                  temp_df['Year'].map(str)

# Summary stats by region
temp_df[['Region', 'Date', 'AvgTemperature']] \
    .groupby(['Region']) \
    .agg(['min', 'max', 'nunique'])
# Note that there are seven regions

Date                   AvgTemperature  \
                                        min       max nunique            min   
Region                                                                         
Africa                             1/1/1995  9/9/2019    9297          -99.0   
Asia                               1/1/1995  9/9/2019    9265          -99.0   
Australia/South Pacific            1/1/1995  9/9/2019    9265          -99.0   
Europe                             1/1/1995  9/9/2019    9356          -99.0   
Middle East                        1/1/1995  9/9/2019    9265          -99.0   
North America                      1/1/1995  9/9/2019    9295          -99.0   
South/Central America & Carribean  1/1/1995  9/9/2019    9266          -99.0   

                                                  
                                     max nunique  
Region                                            
Africa                             102.8     654  
Asia                               103.7    1334  
Australia/South Pacific             96.8     600  
Europe                             102.5    1079  
Middle East                        110.0     996  
North America                      107.7    1474  
South/Central America & Carribean   97.4     609

In [7]:
# Check how consistent information is populated by city over time
temp_df.groupby(['Year'])[['City']].nunique()
# Very consistent going back to 1995

,City
Year,
200,2
201,7
1995,319
1996,319
1997,320
1998,321
1999,321
2000,321
2001,321


In [8]:
# Note: there are records with a day value of 0. Exclude.
temp_df = temp_df[temp_df.Day > 0]

# Note: there are records with a year value of 200 and 201. Exclude.
temp_df = temp_df[temp_df.Year >= 1995]

# Note: missing temps are represented as -99. Exclude.
temp_df = temp_df[temp_df.AvgTemperature != -99]

# Drop duplicates
temp_df = temp_df.drop_duplicates()

# Verify there are no duplicate records at the most granular level
duplicate = temp_df[temp_df.duplicated(['State', 'City', 'Date'], keep=False)] \
                        .sort_values(by = ['City', 'Date'])
duplicate.head(10)
# 94 duplicates remain. Need to research further.

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date
744576,Europe,Germany,NaN,Hamburg,1,25,2011,38.5,1/25/2011
744970,Europe,Germany,NaN,Hamburg,1,25,2011,38.6,1/25/2011
743935,Europe,Germany,NaN,Hamburg,5,25,2010,51.7,5/25/2010
744330,Europe,Germany,NaN,Hamburg,5,25,2010,52.0,5/25/2010
743962,Europe,Germany,NaN,Hamburg,6,21,2010,57.4,6/21/2010
744357,Europe,Germany,NaN,Hamburg,6,21,2010,58.6,6/21/2010
754882,Europe,Germany,NaN,Munich,1,15,2019,35.0,1/15/2019
755249,Europe,Germany,NaN,Munich,1,15,2019,35.1,1/15/2019
754151,Europe,Germany,NaN,Munich,1,16,2018,41.5,1/16/2018
754518,Europe,Germany,NaN,Munich,1,16,2018,40.9,1/16/2018


In [9]:
# Summary stats by city or country for a region of interest.
# Note: there are some time gaps in cities.
def summary_stats(GroupBy, Region):
    return temp_df[[GroupBy, 'Date', 'AvgTemperature']].where(temp_df.Region == Region) \
               .groupby([GroupBy]) \
               .agg(['min', 'max', 'nunique'])

# GroupBy input options: City, Country, State
# Region input options: Africa, Asia, Australia/South Pacific,
#                       Europe, Middle East, North America,
#                       South/Central America & Carribean
summary_stats(GroupBy = 'City', Region = 'South/Central America & Carribean').head()

Date                   AvgTemperature              
                   min       max nunique            min   max nunique
City                                                                 
Belize City   1/1/1996  9/9/2019    8867           64.6  92.9     233
Bogota        1/1/1995  9/9/2019    9196           46.7  66.7     154
Brasilia      1/1/1995  9/9/2019    9206           56.1  87.7     244
Bridgetown    1/1/1995  9/9/2016    8349           74.2  88.0     120
Buenos Aires  1/1/1995  9/9/2019    9231           35.3  90.9     507

In [10]:
# Check if columns contain null values.
temp_df.loc[pd.isnull(temp_df[['Region',
                               'Country',
                               'City',
                               'Date',
                               'AvgTemperature']]).any(1),:]
# Only state has nulls, which is to be expected.

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date


#### Population Counts by Country and Year

In [11]:
# Rename columns to be more descriptive
country_pop_df.columns = ['Country', 'Year', 'Country_Population']

# Check how consistent information is populated by country over time
country_pop_df.groupby(['Year'])[['Country']].nunique()
# Very consistent going back to 1960

,Country
Year,
1960,216
1961,216
1962,216
1963,216
1964,216
1965,216
1966,216
1967,216
1968,216


In [12]:
# Summary stats by country. 
country_pop_df.where(country_pop_df.Country > 'C') \
    .groupby(['Country']) \
    .agg(['min', 'max', 'nunique']).head()

Year                 Country_Population              \
                           min     max nunique                min         max   
Country                                                                         
Cabo Verde              1960.0  2017.0      58           201765.0    537497.0   
Cambodia                1960.0  2017.0      58          5722370.0  16009414.0   
Cameroon                1960.0  2017.0      58          5176918.0  24566045.0   
Canada                  1960.0  2017.0      58         17909009.0  36540268.0   
Caribbean small states  1960.0  2017.0      58          4194710.0   7314990.0   

                                
                       nunique  
Country                         
Cabo Verde                  58  
Cambodia                    58  
Cameroon                    58  
Canada                      58  
Caribbean small states      58

In [13]:
# Verify there are no duplicate records at the most granular level
duplicate = country_pop_df[country_pop_df.duplicated(['Country', 'Year'], keep=False)] \
                .sort_values(by = ['Country', 'Year'])
duplicate.head()
# No duplicates.

,Country,Year,Country_Population


In [14]:
# Check if columns contain null values.
country_pop_df.loc[pd.isnull(country_pop_df[['Country',
                                             'Year',
                                             'Country_Population']]).any(1),:]
# No null values detected.

,Country,Year,Country_Population


#### Population Counts by City and Year

In [15]:
# Filter on sex to only include both sexes since other sources don't include this breakdown.
# Note I verfied there are 4,751 distinct cities and all cities have a both sexes row
city_pop_df[['City']].nunique()
city_pop_df[['Sex','City']].groupby(['Sex']).nunique()
city_pop_df = city_pop_df[city_pop_df.Sex == 'Both Sexes']

# By removing the sex breakdown, the field can be dropped
city_pop_df = city_pop_df.drop(columns=['Sex'])
# city_pop_df.head()

# Check how many inputs are in the Area column
city_pop_df[['Area']].drop_duplicates()

# Remove Area coulmn with there being only one input
city_pop_df = city_pop_df.drop(columns=['Area'])

# Rename columns for naming consistencies
city_pop_df.columns = ['Country_or_Area',
                       'Year',
                       'City',
                       'City_Type',
                       'Record_Type',
                       'Reliability',
                       'Source_Year',
                       'City_Population',
                       'Population_Notes']

# Summary stats by country/area and sex. There are gaps in time.
city_pop_df[['Country_or_Area', 'Year', 'City_Population']] \
    .groupby(['Country_or_Area']) \
    .agg(['min', 'max', 'nunique']).head()

Year               City_Population                   
                  min   max nunique             min        max nunique
Country_or_Area                                                       
Albania          2003  2011       2        113249.0   418495.0       3
Algeria          1998  2008       2        102151.0  2712944.0      78
American Samoa   2000  2010       2          3656.0     4278.0       2
Andorra          2003  2011       8         21245.0    24779.0       8
Anguilla         2001  2011       2          2812.0     4904.0       2

In [16]:
# Drop duplicates
city_pop_df = city_pop_df.drop_duplicates()

# Verify there are no duplicate records at the most granular level
duplicate = city_pop_df[city_pop_df.duplicated(['City', 'Year', 'City_Type'], keep=False)] \
                .sort_values(by = ['City', 'Year'])
duplicate.head()
# There are duplicates. Prioritize most recent source year, record type (census over estimate), and (maybe) reliability.

,Country_or_Area,Year,City,City_Type,Record_Type,Reliability,Source_Year,City_Population,Population_Notes
53989,Spain,2001,A Coruña,City proper,Census - de facto - complete tabulation,"Final figure, complete",2009,236379.0,NaN
53990,Spain,2001,A Coruña,City proper,Estimate - de jure,"Final figure, complete",2002,235847.0,170
52883,Spain,2011,A Coruña,City proper,Census - de jure - complete tabulation,"Final figure, complete",2014,245055.0,74
52884,Spain,2011,A Coruña,City proper,Estimate - de jure,"Final figure, complete",2013,246087.0,74
32822,Kazakhstan,2012,ASTANA,City proper,Estimate - de facto,"Final figure, complete",2018,760506.0,"18,118"


In [17]:
# Check how consistent information is populated by city over time
city_pop_df.groupby(['Year'])[['City']].nunique()
# By year 2000, data collection is much more complete accross cities

,City
Year,
1970,2
1976,9
1980,3
1981,1
1983,9
1984,4
1985,2
1986,2
1987,28


In [18]:
# Review record types and reliability types to familiarize with. 
city_pop_df[['Record_Type', 'Reliability',]] \
    .drop_duplicates() \
    .sort_values(by = ['Record_Type'])
# May have prioirtize when there are multiple sources for the same year and city

,Record_Type,Reliability
157,Census - de facto - complete tabulation,"Final figure, complete"
2678,Census - de facto - complete tabulation,Provisional figure
39,Census - de jure - complete tabulation,"Final figure, complete"
34434,Census - de jure - complete tabulation,Provisional figure
15932,Census - de jure - sample tabulation,"Final figure, complete"
48,Estimate - de facto,"Final figure, complete"
2567,Estimate - de facto,Provisional figure
0,Estimate - de jure,"Final figure, complete"
764,Estimate - de jure,Provisional figure
34484,Estimate - de jure,Other estimate


In [19]:
# Check if columns contain null values. 
city_pop_df.loc[pd.isnull(city_pop_df[['Country_or_Area',
                                       'Year',
                                       'City',
                                       'City_Type',
                                       'Record_Type', 
                                       'Reliability',
                                       'Source_Year', 
                                       'City_Population']]).any(1),:]
# No unexpected nulls.

,Country_or_Area,Year,City,City_Type,Record_Type,Reliability,Source_Year,City_Population,Population_Notes


#### Refugee Counts by Year, Country of Residence, and Country of Origin

In [20]:
# Rename columns for naming consistencies
refugee_df.columns = ['Asylum_Country_or_Territory',
                      'Origin_Country_or_Territory',
                      'Year',
                      'Refugees',
                      'Refugees_Assisted_by_UNHCR',
                      'Refugee-like_Population',
                      'Refugee-like_Population_Assisted_by_UNHCR']

# Drop duplicates
refugee_df = refugee_df.drop_duplicates()

# Get an understanding of the differences between population counts based on summary stats
def pop_stat_comparison(pop1, pop2):
    return refugee_df[['Asylum_Country_or_Territory', pop1, pop2]] \
               .groupby(['Asylum_Country_or_Territory']) \
               .agg(['min', 'max', 'nunique'])

pop_stat_comparison('Refugees', 'Refugees_Assisted_by_UNHCR').head()
# pop_stat_comparison('Refugee-like_Population', 'Refugee-like_Population_Assisted_by_UNHCR')
# pop_stat_comparison('Refugee-like_Population', 'Refugees')

# Takeaways:
# UNHCR related counts are populated substantially less
# Refugees vs. refugee-like: they differ slightly

Refugees                    \
                                 min       max nunique   
Asylum_Country_or_Territory                              
Afghanistan                      1.0  280229.0      29   
Albania                          1.0   22324.0      37   
Algeria                          1.0  165000.0      82   
Angola                           1.0  225000.0     156   
Anguilla                         1.0       1.0       1   

                            Refugees_Assisted_by_UNHCR                    
                                                   min       max nunique  
Asylum_Country_or_Territory                                               
Afghanistan                                        1.0  280229.0      16  
Albania                                            1.0    3918.0      27  
Algeria                                            1.0  155430.0      33  
Angola                                             1.0   13007.0      59  
Anguilla                                           1.0       1.0       1

In [21]:
# Summary stats by country of asylum.
refugee_df[['Asylum_Country_or_Territory', 'Year']] \
    .groupby(['Asylum_Country_or_Territory']) \
    .agg(['min', 'max', 'nunique']).head()
# There are gaps in time.

Year              
                              min   max nunique
Asylum_Country_or_Territory                    
Afghanistan                  1990  2016      24
Albania                      1992  2016      25
Algeria                      1975  2016      42
Angola                       1976  2016      41
Anguilla                     2015  2016       2

In [22]:
# Verify there are no duplicate records at the most granular level
duplicate = refugee_df[refugee_df.duplicated(['Asylum_Country_or_Territory',
                                              'Origin_Country_or_Territory',
                                              'Year'], keep=False)] \
                           .sort_values(by = ['Asylum_Country_or_Territory',
                                              'Origin_Country_or_Territory',
                                              'Year'])
duplicate
# No duplicates

,Asylum_Country_or_Territory,Origin_Country_or_Territory,Year,Refugees,Refugees_Assisted_by_UNHCR,Refugee-like_Population,Refugee-like_Population_Assisted_by_UNHCR


In [23]:
# Check how consistent information is populated by city over time
refugee_df.groupby(['Year'])[['Asylum_Country_or_Territory']].nunique()
# Seems consistent by year, especially considering new countries forming over time

,Asylum_Country_or_Territory
Year,
1975,50
1976,53
1977,72
1978,82
1979,88
1980,90
1981,92
1982,94
1983,97


In [24]:
# Check if columns contain null values. 
refugee_df.loc[pd.isnull(refugee_df[['Asylum_Country_or_Territory',
                                     'Origin_Country_or_Territory',
                                     'Year',
                                     'Refugees',
                                     'Refugee-like_Population']]).any(1),:].head()
# No unexpected nulls. Not all refugee counts populate, which is okay.

,Asylum_Country_or_Territory,Origin_Country_or_Territory,Year,Refugees,Refugees_Assisted_by_UNHCR,Refugee-like_Population,Refugee-like_Population_Assisted_by_UNHCR
2453,Israel,Dem. Rep. of the Congo,2016,NaN,NaN,208.0,50.0
2941,Malaysia,Kenya,2016,NaN,NaN,1.0,1.0
2953,Malaysia,Rep. of Moldova,2016,NaN,NaN,1.0,1.0
2966,Malaysia,United States,2016,NaN,NaN,1.0,1.0
2967,Malaysia,Viet Nam,2016,NaN,NaN,1.0,1.0


#### Verify country and city naming conventions are consistent across data sources

In [25]:
# Pull unique country and city values from each source data set and give them unique names
unique_refugee_origin = refugee_df[['Origin_Country_or_Territory']].drop_duplicates()

unique_refugee_asylum = refugee_df[['Asylum_Country_or_Territory']].drop_duplicates()

unique_citypop_country = city_pop_df[['Country_or_Area']].drop_duplicates()

unique_citypop_country.columns = ['Country_CityPop']

unique_citypop_city = city_pop_df[['Country_or_Area', 'City']].drop_duplicates()

unique_citypop_city.columns = ['Country_CityPop', 'City_CityPop']

unique_countrypop = country_pop_df[['Country']].drop_duplicates()

unique_countrypop.columns = ['Country_CountryPop']

unique_temp_country = temp_df[['Country']].drop_duplicates()

unique_temp_country.columns = ['Country_Temp']

unique_temp_city = temp_df[['Country', 'State', 'City']].drop_duplicates()

unique_temp_city.columns = ['Country_Temp', 'State_Temp', 'City_Temp']

In [26]:
# Source: https://github.com/konstantinstadler/country_converter
countrypop_country_list = unique_countrypop['Country_CountryPop'].tolist()
countrypop_standard_names = coco.convert(names=countrypop_country_list, to='name_short')
# print(countrypop_standard_names)
unique_countrypop['CountryStandard'] = countrypop_standard_names
unique_countrypop.head()

Channel Islands not found in regex
Caribbean small states not found in regex
Pacific island small states not found in regex


,Country_CountryPop,CountryStandard
0,Aruba,Aruba
1,Afghanistan,Afghanistan
2,Angola,Angola
3,Albania,Albania
4,Andorra,Andorra


In [27]:
# Replace values not found with source value
unique_countrypop.CountryStandard[unique_countrypop.CountryStandard == 'not found'] = unique_countrypop.Country_CountryPop

In [28]:
unique_temp_country.Country_Temp[unique_temp_country.Country_Temp == 'Equador'] = 'Ecuador'
temp_country_list = unique_temp_country['Country_Temp'].tolist()
temp_standard_names = coco.convert(names=temp_country_list, to='name_short')
unique_temp_country['CountryStandard'] = temp_standard_names
unique_temp_country.head()
# print(temp_standard_names)

Serbia-Montenegro not found in regex
Yugoslavia not found in regex


,Country_Temp,CountryStandard
0,Algeria,Algeria
9269,Burundi,Burundi
13809,Benin,Benin
23075,Central African Republic,Central African Republic
32341,Congo,Congo Republic


In [29]:
# Replace values not found with source value
unique_temp_country.CountryStandard[unique_temp_country.CountryStandard == 'not found'] = unique_temp_country.Country_Temp

In [30]:
citypop_country_list = unique_citypop_country['Country_CityPop'].tolist()
citypop_standard_names = coco.convert(names=citypop_country_list, to='name_short')
unique_citypop_country['CountryStandard'] = citypop_standard_names
unique_citypop_country.head()
# print(citypop_standard_names)

,Country_CityPop,CountryStandard
0,Åland Islands,Aland Islands
42,Albania,Albania
51,Algeria,Algeria
129,American Samoa,American Samoa
133,Andorra,Andorra


In [31]:
# Replace values not found with source value
unique_citypop_country.CountryStandard[unique_citypop_country.CountryStandard == 'not found'] = unique_citypop_country.Country_CityPop

In [32]:
unique_refugee_asylum.Asylum_Country_or_Territory[unique_refugee_asylum.Asylum_Country_or_Territory == 'Serbia (and Kosovo: S/RES/1244 (1999))'] = 'Serbia'
refugee_AsylumCountry_list = unique_refugee_asylum['Asylum_Country_or_Territory'].tolist()
refugee_Asylum_standard_names = coco.convert(names=refugee_AsylumCountry_list, to='name_short')
unique_refugee_asylum['CountryStandard'] = refugee_Asylum_standard_names
unique_refugee_asylum.head()
# print(refugee_Asylum_standard_names)

Various not found in regex


,Asylum_Country_or_Territory,CountryStandard
0,Afghanistan,Afghanistan
3,Albania,Albania
18,Algeria,Algeria
39,Angola,Angola
66,Anguilla,Anguilla


In [33]:
unique_refugee_origin.Origin_Country_or_Territory[unique_refugee_origin.Origin_Country_or_Territory == 'Serbia (and Kosovo: S/RES/1244 (1999))'] = 'Serbia'
refugee_OriginCountry_list = unique_refugee_origin['Origin_Country_or_Territory'].tolist()
refugee_Origin_standard_names = coco.convert(names=refugee_OriginCountry_list, to='name_short')
unique_refugee_origin['CountryStandard'] = refugee_Origin_standard_names
unique_refugee_origin.head()
# print(refugee_Origin_standard_names)

Stateless not found in regex
Various not found in regex
Tibetans not found in regex


,Origin_Country_or_Territory,CountryStandard
0,Iraq,Iraq
1,Islamic Rep. of Iran,Iran
2,Pakistan,Pakistan
3,China,China
4,Dem. Rep. of the Congo,DR Congo


In [34]:
# Replace values not found with source value
unique_refugee_asylum.CountryStandard[unique_refugee_asylum.CountryStandard == 'not found'] = unique_refugee_asylum.Asylum_Country_or_Territory
unique_refugee_origin.CountryStandard[unique_refugee_origin.CountryStandard == 'not found'] = unique_refugee_origin.Origin_Country_or_Territory

In [35]:
# Full outer join coutry population and temperature dataframes for mapping purposes
# Source: https://www.analyticsvidhya.com/blog/2020/02/joins-in-pandas-master-the-different-types-of-joins-in-python/
country_map = pd.merge(unique_countrypop, unique_temp_country, on='CountryStandard', how='outer')

In [36]:
# Full outer join city population and previously defined map dataframes
country_map = pd.merge(country_map, unique_citypop_country, on='CountryStandard', how='outer')

In [37]:
# Full outer join asylum refugees and previously defined map dataframes
country_map = pd.merge(country_map, unique_refugee_asylum, on='CountryStandard', how='outer')

In [38]:
# Full outer join origin refugees and previously defined map dataframes
country_map = pd.merge(country_map, unique_refugee_origin, on='CountryStandard', how='outer')
country_map.head()

,Country_CountryPop,CountryStandard,Country_Temp,Country_CityPop,Asylum_Country_or_Territory,Origin_Country_or_Territory
0,Aruba,Aruba,NaN,Aruba,Aruba,NaN
1,Afghanistan,Afghanistan,NaN,NaN,Afghanistan,Afghanistan
2,Angola,Angola,NaN,NaN,Angola,Angola
3,Albania,Albania,Albania,Albania,Albania,Albania
4,Andorra,Andorra,NaN,Andorra,NaN,Andorra


In [39]:
# Create state and abbreviation mapping dataframe
# Source: https://github.com/unitedstates/python-us
state = us.states.mapping('abbr', 'name')
# state.keys()
# state.values()
state_abbr = pd.DataFrame(list(state.items()))
state_abbr.columns = ['StateAbbreviation', 'State']
state_abbr.head(20)

,StateAbbreviation,State
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California
5,CO,Colorado
6,CT,Connecticut
7,DE,Delaware
8,FL,Florida
9,GA,Georgia


In [40]:
# Add country standard and state info to both datasets containing cities
unique_temp_city = sqldf("""
    SELECT DISTINCT 
        country_map.CountryStandard,
        state_abbr.StateAbbreviation,
        temp.*
    FROM unique_temp_city as temp
    LEFT JOIN country_map
        ON country_map.Country_Temp = temp.Country_Temp
    LEFT JOIN state_abbr
        ON temp.State_Temp = state_abbr.State
    """)

unique_citypop_city = sqldf("""
    SELECT DISTINCT 
        country_map.CountryStandard,
        State, 
        StateAbbreviation,
        CityPop.*
    FROM unique_citypop_city AS CityPop
    LEFT JOIN country_map
        ON country_map.Country_CityPop = CityPop.Country_CityPop
    LEFT JOIN state_abbr
        ON SUBSTR(CityPop.City_CityPop, length(CityPop.City_CityPop) - 2, 2) = state_abbr.StateAbbreviation
            AND country_map.CountryStandard = 'United States'
    """)

unique_citypop_city.head()

,CountryStandard,State,StateAbbreviation,Country_CityPop,City_CityPop
0,Aland Islands,None,None,Åland Islands,MARIEHAMN
1,Albania,None,None,Albania,Durrës
2,Albania,None,None,Albania,TIRANA
3,Algeria,None,None,Algeria,Adrar
4,Algeria,None,None,Algeria,Ain Defla


In [41]:
# Build mapping for state, city, and country standards
# The termperature source joins to the city poulation source several
# time to populate the city field as best as possible without too much hardcoding.
temp_city_map = sqldf("""
    SELECT DISTINCT 
        unique_temp_city.CountryStandard as CountryStandard_Temp,
        unique_temp_city.Country_Temp,
        --unique_citypop_city.Country_CityPop,
        unique_temp_city.State_Temp as StateStandard_Temp,
        unique_temp_city.State_Temp,
        --COALESCE(CityUS.State, CityUSPartial.State) as State_CityPop,
        unique_temp_city.City_Temp as CityStandard,
        unique_temp_city.City_Temp,
        -- unique_citypop_city.City_CityPop,
        -- CityUS.City_CityPop as CityUS,
        -- CityUSPartial.City_CityPop as CityUSPartial,
        -- partial6.City_CityPop as CityPartial6,
        -- partial5.City_CityPop as CityPartial5,
        -- partial4.City_CityPop as CityPartial4,
        -- partial3.City_CityPop as CityPartial3,
        COALESCE(unique_citypop_city.City_CityPop, CityUS.City_CityPop, CityUSPartial.City_CityPop, 
                 partial6.City_CityPop, partial5.City_CityPop, partial4.City_CityPop, partial3.City_CityPop) AS City_CityPop
    FROM unique_temp_city
    -- Join on city (case protected) and country
    LEFT JOIN unique_citypop_city
        ON UPPER(unique_citypop_city.City_CityPop) = UPPER(unique_temp_city.City_Temp)
            AND unique_citypop_city.CountryStandard = unique_temp_city.CountryStandard
    -- Strip the US abbreviation at the end of the city 
    -- and join on city, state, and country (US only)
    LEFT JOIN unique_citypop_city AS CityUS
        ON SUBSTR(UPPER(CityUS.City_CityPop),1,length(CityUS.City_CityPop) - 5) = UPPER(unique_temp_city.City_Temp)
            AND CityUS.CountryStandard = unique_temp_city.CountryStandard
            AND CityUS.State = unique_temp_city.State_Temp
            AND unique_temp_city.CountryStandard = 'United States'
            AND unique_citypop_city.City_CityPop IS NULL
    -- Join on city matching the first five characters, state, and country (US only)
    LEFT JOIN unique_citypop_city AS CityUSPartial
        ON SUBSTR(UPPER(CityUSPartial.City_CityPop),1,5) = SUBSTR(UPPER(unique_temp_city.City_Temp),1,5)
            AND CityUSPartial.CountryStandard = unique_temp_city.CountryStandard
            AND CityUSPartial.State = unique_temp_city.State_Temp
            AND unique_temp_city.CountryStandard = 'United States'
            AND unique_citypop_city.City_CityPop IS NULL
            AND CityUS.City_CityPop IS NULL
    -- Join on city matching the first six characters, state, and country (non US)
    LEFT JOIN unique_citypop_city AS partial6
        ON SUBSTR(UPPER(partial6.City_CityPop),1,6) = SUBSTR(UPPER(unique_temp_city.City_Temp),1,6)
            AND partial6.CountryStandard = unique_temp_city.CountryStandard
            AND unique_temp_city.CountryStandard <> 'United States'
            AND unique_citypop_city.City_CityPop IS NULL
            AND CityUS.City_CityPop IS NULL
            AND CityUSPartial.City_CityPop IS NULL
    -- Join on city matching the first five characters, state, and country (non US)
    LEFT JOIN unique_citypop_city AS partial5
        ON SUBSTR(UPPER(partial5.City_CityPop),1,5) = SUBSTR(UPPER(unique_temp_city.City_Temp),1,5)
            AND partial5.CountryStandard = unique_temp_city.CountryStandard
            AND unique_temp_city.CountryStandard <> 'United States'
            AND unique_citypop_city.City_CityPop IS NULL
            AND CityUS.City_CityPop IS NULL
            AND CityUSPartial.City_CityPop IS NULL
            AND partial6.City_CityPop IS NULL
    -- Join on city matching the first four characters, state, and country (non US)
    LEFT JOIN unique_citypop_city AS partial4
        ON SUBSTR(UPPER(partial4.City_CityPop),1,4) = SUBSTR(UPPER(unique_temp_city.City_Temp),1,4)
            AND partial4.CountryStandard = unique_temp_city.CountryStandard
            AND unique_temp_city.CountryStandard <> 'United States'
            AND unique_citypop_city.City_CityPop IS NULL
            AND CityUS.City_CityPop IS NULL
            AND CityUSPartial.City_CityPop IS NULL
            AND partial6.City_CityPop IS NULL
            AND partial5.City_CityPop IS NULL
    -- Join on city matching the first three characters, state, and country (non US)
    LEFT JOIN unique_citypop_city AS partial3
        ON SUBSTR(UPPER(partial3.City_CityPop),1,3) = SUBSTR(UPPER(unique_temp_city.City_Temp),1,3)
            AND partial3.CountryStandard = unique_temp_city.CountryStandard
            AND unique_temp_city.CountryStandard <> 'United States'
            AND partial3.City_CityPop NOT IN ('BELMOPAN', 'Bommanahalli', 'Brugge')
            AND unique_citypop_city.City_CityPop IS NULL
            AND CityUS.City_CityPop IS NULL
            AND CityUSPartial.City_CityPop IS NULL
            AND partial6.City_CityPop IS NULL
            AND partial5.City_CityPop IS NULL
            AND partial4.City_CityPop IS NULL
    --WHERE unique_citypop_city.City_CityPop IS NULL 
    --    and CityUS is null
    --    and CityUSPartial is not null
      --  and partial6.City_CityPop IS NULL
      --  and partial5.City_CityPop IS NULL
      --  and partial4.City_CityPop IS NULL
      --  and partial3.City_CityPop IS not NULL
    ORDER BY unique_temp_city.City_Temp
    """)

# 109/321 link on initial join
# 211/321 link with second join added
# 235/321 link with third join added
# 244/321 link with fourth join added
# 249/321 link with fifth join added
# 253/321 link with sixth join added
# 264/321 link with seventh join added

In [47]:
# Join mapping to city population set to have standard naming across
citypop_city_map = sqldf("""
    SELECT DISTINCT 
        CityPop.CountryStandard as CountryStandard_CityPop,
        COALESCE(temp_city_map.StateStandard_Temp, CityPop.State) as StateStandard_CityPop,
        COALESCE(temp_city_map.CityStandard, CityPop.City_CityPop) as CityStandard, 
        CityPop.Country_CityPop,
        CityPop.State as State_CityPop,
        CityPop.City_CityPop
    FROM unique_citypop_city AS CityPop
    LEFT JOIN temp_city_map
        ON CityPop.City_CityPop = temp_city_map.City_CityPop
            AND CityPop.CountryStandard = temp_city_map.CountryStandard_Temp
    """)

# Propercase the city standard, so it's clean in presentation
citypop_city_map["CityStandard"] = citypop_city_map["CityStandard"].str.title()

# Drop city name that is a duplicate when the two mapping tables merge in the following step
temp_city_map = temp_city_map.drop(columns=['City_CityPop'])

In [48]:
# Full outer join on city maps
city_map = pd.merge(temp_city_map, citypop_city_map, on='CityStandard', how='outer')
city_map.head()

,CountryStandard_Temp,Country_Temp,StateStandard_Temp,State_Temp,CityStandard,City_Temp,CountryStandard_CityPop,StateStandard_CityPop,Country_CityPop,State_CityPop,City_CityPop
0,Cote d'Ivoire,Ivory Coast,None,None,Abidjan,Abidjan,Cote d'Ivoire,None,Côte d'Ivoire,None,Abidjan
1,United States,US,Texas,Texas,Abilene,Abilene,United States,Texas,United States of America,Texas,Abilene (TX)
2,United Arab Emirates,United Arab Emirates,None,None,Abu Dhabi,Abu Dhabi,NaN,NaN,NaN,NaN,NaN
3,Ethiopia,Ethiopia,None,None,Addis Ababa,Addis Ababa,Ethiopia,None,Ethiopia,None,ADDIS ABABA
4,United States,US,Ohio,Ohio,Akron Canton,Akron Canton,United States,Ohio,United States of America,Ohio,Akron (OH)


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The data warehouse will reflect a relational data model with a star schema. The relational model complements the use case of analytical processes, especially with expected changes in business requirements over time. Utilizing a dimensional model gives end users an intuitive layout, the flexibility to use SQL, and high data integrity.

Fact Tables
 1. **TemperatureFact** - records from the temperature dataset associated with daily average temperatures in cities around the world
     - TemperatureKey, AvgTemperature, DateKey, CountryKey, CityKey
 2. **CountryPopulationFact** - records from the country population dataset reflecting population counts by year and country
     - CountryPopulationKey, CountryPopulation,  CountryKey
 3. **CityPopulationFact** - records from the city population dataset reflecting population counts by year and city
     - CityPopulationKey, CityPopulation, SourceKey, CityKey,  CountryKey
 4. **RefugeeFact** - records from the refugee dataset associated with refugee and refugee-like populations by country and year
     - RefugeeKey,  RefugeePopulation, RefugeesAssistedByUNHCR, RefugeeLikePopulation, RefugeeLikesAssistedByUNHCR, AsylumCountryKey, OriginCountryKey

Dimension Tables
 1. **DateDim** - dates of temperature recordings
     - DateKey, Date, Month, Day, Year
 2. **CountryDim** - country, year, and region content in climate refugee database
     - CountryKey, Country, Region, Year
 3. **CityDim** - city, year, and state content in climate refugee database
     - CityKey, City, CityType, State, Year
 4. **SourceDim** - source details in climate refugee database
     - SourceKey, SourceYear, CityPopulationNotes, RecordType, Reliability

#### 3.2 Mapping Out Data Pipelines

**Create Table Schemas Based on Conceptual Model**
 1. Write create table and drop table statements
 2. Add logic to connect the tables to the Amazon Redshift database
 3. Launch a Redshift cluster and attach an IAM role with S3 read access
 4. Add the cluster and IAM role content to dwh.cfg
 5. Test execution by verifying the empty tables exist in the Redshift database
 
**Build ETL Pipeline**
 1. Create an S3 bucket and load the source data sets into S3
 2. Stage source datasets from S3 into the analytics database
 3. Transform the staged datasets into tables reflecting the conceptual model
 4. Load the tables into the analytics database
 5. Perform data quality checks and revise accordingly as needed
 6. Once finished, delete the redshift cluster

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

In [85]:
# Table list used to drop tables
table_list = ['staging_temperatures',
              'staging_country_populations',
              'staging_city_populations',
              'staging_refugees',
              'TemperatureFact',
              'CountryPopulationFact',
              'CityPopulationFact',
              'RefugeeFact',
              'DateDim',
              'CountryDim',
              'CityDim',
              'SourceDim']

def drop_tables(cur, conn):
    """
    Drops each table if they exist for each table in table_list
    
    INPUTS:
    * cur - the cursor available
    * conn - database connection
    """
    for table in table_list:
        cur.execute("DROP TABLE IF EXISTS " + table)
        conn.commit()

In [87]:
# Create tables
create_staging_temperatures = ("""CREATE TABLE IF NOT EXISTS staging_temperatures(
                                      Region varchar NOT NULL distkey,
                                      Country varchar NOT NULL,
                                      State varchar,
                                      City varchar NOT NULL,
                                      Month int NOT NULL,
                                      Day int NOT NULL,
                                      Year int NOT NULL,
                                      AvgTemperature decimal NOT NULL,
                                      Date date NOT NULL)""")

create_staging_country_populations = ("""CREATE TABLE IF NOT EXISTS staging_country_populations(
                                             Country varchar NOT NULL distkey,
                                             Year int NOT NULL,
                                             CountryPopulation int NOT NULL)""")

create_staging_city_populations = ("""CREATE TABLE IF NOT EXISTS staging_city_populations(
                                          CoutryOrArea varchar NOT NULL distkey,
                                          Year int NOT NULL,
                                          City varchar NOT NULL,
                                          RecordType varchar NOT NULL,
                                          Reliability varchar NOT NULL,
                                          SourceYear int NOT NULL,
                                          CityPopulation int NOT NULL,
                                          PopulationNotesKey varchar NOT NULL)""")

create_staging_refugees = ("""CREATE TABLE IF NOT EXISTS staging_refugees(
                                  AsylumCoutryOrTerritory varchar NOT NULL distkey,
                                  OriginCoutryOrTerritory varchar NOT NULL,
                                  Year int NOT NULL,
                                  RefugeePopulation int,
                                  RefugeesAssistedByUNHCR int,
                                  RefugeeLikePopulation int,
                                  RefugeeLikesAssistedByUNHCR int)""")

create_TemperatureFact = ("""CREATE TABLE IF NOT EXISTS TemperatureFact(
                                 TemperatureKey int IDENTITY(0,1),
                                 DateKey int NOT NULL REFERENCES DateDim sortkey,
                                 CountryKey int NOT NULL REFERENCES CountryDim,
                                 CityKey int NOT NULL REFERENCES CityDim,
                                 AvgTemperature decimal NOT NULL,
                                 PRIMARY KEY(TemperatureKey))""")

create_CountryPopulationFact = ("""CREATE TABLE IF NOT EXISTS CountryPopulationFact(
                                       CountryPopulationKey int IDENTITY(0,1),
                                       CountryPopulation int NOT NULL,
                                       CountryKey int NOT NULL REFERENCES CountryDim sortkey,
                                       PRIMARY KEY(CountryPopulationKey))""")

create_CityPopulationFact = ("""CREATE TABLE IF NOT EXISTS CityPopulationFact(
                                    CityPopulationKey int IDENTITY(0,1),
                                    CityPopulation int NOT NULL,
                                    CityKey int NOT NULL REFERENCES CityDim sortkey,
                                    CountryKey int NOT NULL REFERENCES CountryDim,
                                    SourceKey int NOT NULL REFERENCES SourceDim,
                                    PRIMARY KEY(CityPopulationKey))""")

create_RefugeeFact = ("""CREATE TABLE IF NOT EXISTS RefugeeFact(
                             RefugeeKey int IDENTITY(0,1),
                             RefugeePopulation int,
                             RefugeesAssistedByUNHCR int,
                             RefugeeLikePopulation int,
                             RefugeeLikesAssistedByUNHCR int,
                             AsylumCountryKey int NOT NULL REFERENCES CountryDim sortkey,
                             OriginCountryKey int NOT NULL REFERENCES CountryDim,
                             PRIMARY KEY(RefugeeKey))""")

create_DateDim = ("""CREATE TABLE IF NOT EXISTS DateDim(
                         DateKey int sortkey,
                         Date date,
                         Month int,
                         Day int,
                         Year int,
                         PRIMARY KEY(DateKey))
                         diststyle all""")

create_CountryDim = ("""CREATE TABLE IF NOT EXISTS CountryDim(
                            CountryKey int sortkey,
                            Country varchar NOT NULL,
                            Region varchar NOT NULL,
                            Year int NOT NULL,
                            PRIMARY KEY(CountryKey))
                            diststyle all""")

create_CityDim = ("""CREATE TABLE IF NOT EXISTS CityDim(
                            CityKey int sortkey,
                            City varchar NOT NULL,
                            CityType varchar NOT NULL,
                            State varchar,
                            Year int NOT NULL,
                            PRIMARY KEY(CityKey))
                            diststyle all""")

create_SourceDim = ("""CREATE TABLE IF NOT EXISTS SourceDim(
                           SourceKey int sortkey,
                           Reliability varchar NOT NULL,
                           RecordType varchar NOT NULL,
                           CityPopulationNotes varchar,
                           SourceYear int NOT NULL,
                           PRIMARY KEY(SourceKey))
                           diststyle all""")

# Create table query list
create_table_queries = [create_staging_temperatures,
                        create_staging_country_populations,
                        create_staging_city_populations,
                        create_staging_refugees,
                        create_DateDim,
                        create_CountryDim,
                        create_CityDim,
                        create_SourceDim,
                        create_TemperatureFact,
                        create_CountryPopulationFact,
                        create_CityPopulationFact,
                        create_RefugeeFact]

def create_tables(cur, conn):
    """
    Creates each table if they don't already exist by executing the \
    queries in `create_table_queries` list from sql_queries.py
    
    INPUTS:
    * cur - the cursor available
    * conn - database connection
    """
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [88]:
# Read in parameters needed for Redshift cluster
config = configparser.ConfigParser()
config.read('dwh.cfg')

# Connect to Redshift cluster and gets cursor to it
conn = psycopg2.connect("""host={} 
                           dbname={} 
                           user={} 
                           password={} 
                           port={}""".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [89]:
# Drops all tables by calling the drop_tables function
drop_tables(cur, conn)

# Creates all tables by calling the create_tables function
create_tables(cur, conn)

In [84]:
# Close the connection
conn.close()

In [ ]:
# Transform the source datasets to have standardized names


In [94]:
# Stage source datasets from dataframe into the analytics database
# df.to_sql('your_table', conn, index=False, if_exists='replace')

,Country,Year,Country_Population
0,Aruba,1960,54211
1,Afghanistan,1960,8996973
2,Angola,1960,5454933
3,Albania,1960,1608800
4,Andorra,1960,13411


In [ ]:
# Transform the staged datasets into tables reflecting the conceptual model

In [ ]:
# Load the tables into the analytics database

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from.